<a href="https://colab.research.google.com/github/dkbhatta/MScFE690-Capstone-PrjGrp-7121/blob/main/detect_NR7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#
#This notebook takes a stock ticker input downloads historical data using yFinance and then uses the data to identify dates where NR7 (narrowest range of 7 days) is idetified
#

In [ ]:
%%capture
!pip install yfinance

In [2]:
from warnings import filterwarnings
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import plotly.offline as pyo

filterwarnings('ignore')

In [3]:
def fetch_data(ticker,start,end):
  data = yf.download(ticker,start,end,progress=False)
  return data

In [4]:
def detect_NR7(data):
  data['Range'] = data['High'] - data['Low']
  data['NR7'] = data['Range'].rolling(window=7).min()
  nr7_days=[]
  for index, row in data.iterrows():
    # Check if row['NR7'] is not NaN before comparing to avoid the error
    if pd.notna(row['NR7']) and row['Range'] == row['NR7']:
      nr7_days.append(index)
  return nr7_days

In [5]:
def plot_NR7(data,nr7_days):
  """

  Args:
    data: historical data for the stock
    nr7_days: list of dates where the narrowest range of 7 days was identified
  """
  # prompt: plot the data indicating nr7 dates
  # Assuming 'data' and 'nr7_days' are already defined from the previous code

  fig = go.Figure(data=[go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

  # Add markers for NR7 days
  for day in nr7_days:
      fig.add_shape(type="line",
                x0=day.strftime('%Y-%m-%d'),
                y0=data.loc[day.strftime('%Y-%m-%d')]['Low'],
                x1=day.strftime('%Y-%m-%d'),
                y1=data.loc[day.strftime('%Y-%m-%d')]['High'],
                line=dict(color="Purple", width=8, dash="dot",))

  fig.update_layout(title='Stock Price with NR7 Days Marked')
  pyo.plot(fig)
  fig.show()
  return

In [9]:
def main():
  ticker = input("Enter the stock ticker: ")
  start = input("Enter the start date (YYYY-MM-DD): ")
  end = input("Enter the end date (YYYY-MM-DD): ")
  print(f"Fetching data for {ticker} from {start} to {end}")
  data = fetch_data(ticker,start,end)
  nr7_days = detect_NR7(data)
  ##print(f"The following days had the narrowest range of 7 days: {nr7_days}")
  print(f"The following dates had the narrowest range of 7 days:")
  for day in nr7_days:
    print(day.strftime('%Y-%m-%d'))
  plot_NR7(data,nr7_days)


if __name__ == "__main__":
  main()

#

Enter the stock ticker: HUT
Enter the start date (YYYY-MM-DD): 2024-09-01
Enter the end date (YYYY-MM-DD): 2024-12-02
Fetching data for HUT from 2024-09-01 to 2024-12-02


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().